# Notebook 7 - Crear y consultar una Base de Datos no relacional orientada a Documentos (Mongo)

## Ejercicio 1: Primeros pasos desde Mongo CLI

En un terminal, verificar si el SGBD Mongo ya está iniciado:

<code>ps -aux | grep mongod</code>

Si no está iniciado, iniciarlo con el comando: <code>mongod</code>

Luego, abrir otra pestaña de terminal (CTRL + Shift + T) e iniciar el cliente CLI de mongo: <code>mongo</code>.

1) Creamos una nueva base de datos llamado <b>nb7-peliculas</b>

<code>use nb7-peliculas</code>

Deberian ver como respuesta del sistema la respuesta: "switched to db nb7-peliculas"

Mongo CLI no utiliza SQL para realizar operaciones CRUD (Create/Read/Update/Delete), utiliza su propia API parecida al lenguaje Javascript.

La variable <code>db</code> representa siempre la base de datos que estamos utilizando (en este caso "nb7-peliculas").

2) Creamos una colección <b>actors</b>

<code>db.actors.insert({firstname:"Will", lastname:"Smith"})</code>

Como pueden verlo, para crear la colección no tuvimos que definir la estructura de la colección, simplemente hemos insertado un documento.

Verificamos que hemos realmente creado la coleccion con:

<code>db.actors.find()</code>

3) Insertamos varios documentos en la colección en un solo comando:

<code>db.actors.insertMany([{firstname:"Scarlett", lastname:"Johanson"},{firstname:"Matt", lastname:"Damon"},{firstname:"Nathan", lastname:"Smith"}])</code>

4) Contamos cuántos documentos tenemos en la colección:

<code>db.actors.count()</code>

Como en SQL, la API de Mongo permite construir consultas para filtrar, ordenar o agrupar datos. Sin embargo, la síntaxis es distinta (y amenudo menos simple e intuitiva que SQL).

5) Filtramos los actores que se llaman 'Smith'

<code>db.actors.find({lastname:"Smith"})</code>

Mongo tambien permite modificar los datos (ojo: Mongo no tiene mecanismo interno para asegurar la coherencia de los datos, es la responsabilidad del programador de la aplicación).

6) Actualizamos un documento. Por ejemplo, "Nathan Smith" se llama ahora "Nathan Lane"

<code>db.actors.update({firstname:"Nathan", lastname:"Smith"},{$set : {lastname:"Lane"}})</code>

Este último comando va a actualizar el primer documento encontrado que corresponde al filtro definido.

7) Supongamos ahora que queremos agregar un campo "gender" en todos los documentos:

¿Qué hace el comando siguiente? ¿Cuál es el limite?

<code>db.actors.update({},{$set : {gender:"M"}})</code>

Para actualizar todos los documentos, y no simplemente el primero, podemos utilizar el comando:

<code>db.actors.update({},{$set : {gender:"M"}},{multi:true})</code>

8) Actualizar el documento "Scarlett Johanson", para indicar que es una mujer (gender:"F").

9) Para suprimir un atributo, podemos utilizar el operador $unset. Por ejemplo:
    
<code>db.actors.update({},{$unset : {gender:1}},{multi:true})</code>

10) Para suprimir un documento completo, podemos utilizar el comando:

<code>db.actors.remove({lastname:"Smith"})</code>

11) Agregamos un atributo de tipo arreglo en un documento

<code>db.actors.update({firstname:"Matt"},{$set : {tab:[1,"a"]}})</code>

12) Añadimos un valor en el arreglo con el operator $push

<code>db.actors.update({firstname:"Matt"},{$push : {tab:["new_value"]}})</code>

¿Cuál es el problema con el último comando? ¿Cuál es la diferencia con el comando siguiente?

<code>db.actors.update({firstname:"Matt"},{$push : {tab:"new_value"}})</code>

Cómo pueden verlo el SGBD no relacional Mongo es mucho más flexible que MySQL, pero deja mucho más posibilidad de equivocarse.


No duden en consultar la documentación de Mongo para probar otros comandos CRUD: https://docs.mongodb.com/manual/crud/


## Ejercicio 2: Primeras consultas avanzadas con una colección de peliculas

En este segundo ejercicio, trabajaremos con una base de datos existente llamada "movie details", la cual tiene 2295 documentos JSON resumiendo información sobre peliculas.

- Descargar el dataset https://raw.githubusercontent.com/steveren/docs-assets/charts-tutorial/movieDetails.json. ¿Qué contiene este archivo? Un documento JSON? Un arreglo de documentos JSON? Una lista de documentos JSON?

- Importamos los datos en Mongo con el comando siguiente:

<code>mongoimport --db nb7-peliculas --collection movieDetails --drop --file <path/to/movieDetails.json></code>

A continuación, queremos realizar algunas consultas típicas para observar la diferencia entre el lenguaje de consulta SQL y el lenguaje de Mongo.

<b>Nota bena</b>: ¿Qué pasa si agregan la función <code>.pretty()</code> al final de un comando? Por ejemplo: <code>db.actors.find().pretty()</code>

1) Utilizar un comando Mongo para contar cuántos documentos hay en la colección 'movieDetails'. ¿Cuál sería el equivalente en SQL?

<code>select count(*) from movieDetails</code>/<code>db.movieDetails.count()</code>

2) Mostrar los títulos (title) y años de producción (year) de todas las peliculas de la colección. ¿A qué clausula SQL correspondría?

<code>select title, year from movieDetails</code>/<code>db.movieDetails.find({},{title:1,year:1})</code>

3) Contar las peliculas realizadas entre 1990 y 2000

<code>db.movieDetails.count({$and:[{year:{$gte:1990}},{year:{$lte:2000}}]})</code>

4) ¿Cuáles son los titulos de peliculas que ganaron más de 5 awards?

<code>db.movieDetails.find({"awards.wins":{$gt:5}},{title:1,"awards.wins":1,_id:0})</code>

5) ¿Cuál es el rating promedio en el sitio imdb para todas las peliculas?

<code>db.movieDetails.aggregate([{$group:{_id:null,average:{$avg:"$imdb.rating"}}}])</code>

6) ¿Cuáles son las peliculas en las cuales participó el actor Nicolas Cage?

<code>db.movieDetails.find({actors:"Nicolas Cage"},{_id:0,title:1})</code>

7) ¿Cuál es el número de pelicula por año?

<code>db.movieDetails.aggregate([{$group:{_id:"$year",numeropeliculas:{$sum:1}}}])</code>

8) ¿Cuál es la pelicula con el mejor rating?

<code>db.movieDetails.find({},{title:1,"imdb.rating":1,_id:0}).sort({"imdb.rating":-1}).limit(1)</code>

9) ¿Cuáles son los thrillers con un rating de al menos 6?

<code>db.movieDetails.find({$and:[{"imdb.rating":{$gte:6}},{genres:"Thriller"}]},{_id:0,title:1,"imdb.rating":1})</code>

10) ¿Cuáles son todos los valores de generos posibles?

<code>db.movieDetails.distinct("genres")</code>

11) ¿Cuáles son las peliculas que contienen la palabra 'Star'?

<code>db.movieDetails.find({title: {$regex:".*star", $options:"i"}},{_id:0,title:1})</code>

12) ¿Cuáles son los títulos que empiezan con la letra 'T'?

<code>db.movieDetails.find({title:{$regex:/^T/}},{title:1})</code>

13) ¿Cuáles son las peliculas que no son de tipo Drama o Thriller?

<code>db.movieDetails.find({$and:[{genres:{$ne:"Thriller"}},{genres:{$ne:"Drama"}}]})</code>

## Ejercicio 3: Integrar Mongo con Python

Desde un entorno Python, conectarse a su base de datos movieDetails y realizar consultas similares al ejercicio 2.

In [98]:
import json
import pymongo
from pymongo import MongoClient
from pprint import pprint

client = MongoClient('localhost', 27017)
db = client['nb7_peliculas']
collection = db['movieDetails']

In [99]:
#1) Utilizar un comando Mongo para contar cuántos documentos hay en la colección 'movieDetails'

count = db.movieDetails.estimated_document_count()
print(count)

2295


In [100]:
#2) Mostrar los títulos (title) y años de producción (year) de todas las peliculas de la colección

for i in db.movieDetails.find({},{'title':1,'year':1,'_id':0}):
    print(i)

{'title': 'West Side Story', 'year': 1961.0}
{'title': 'Wild Wild West', 'year': 1999.0}
{'title': 'Once Upon a Time in the West', 'year': 1968.0}
{'title': 'An American Tail: Fievel Goes West', 'year': 1991.0}
{'title': 'Red Rock West', 'year': 1993.0}
{'title': 'How the West Was Won', 'year': 1962.0}
{'title': 'Slow West', 'year': 2015.0}
{'title': 'A Million Ways to Die in the West', 'year': 2014.0}
{'title': 'Journey to the West', 'year': 2013.0}
{'title': 'Star Wars: Episode V - The Empire Strikes Back', 'year': 1980.0}
{'title': 'West of Memphis', 'year': 2012.0}
{'title': 'Star Wars: Episode IV - A New Hope', 'year': 1977.0}
{'title': 'Star Wars: Episode I - The Phantom Menace', 'year': 1999.0}
{'title': 'Star Wars: Episode VI - Return of the Jedi', 'year': 1983.0}
{'title': 'Star Wars: Episode III - Revenge of the Sith', 'year': 2005.0}
{'title': 'Star Wars: Episode II - Attack of the Clones', 'year': 2002.0}
{'title': 'Star Trek Into Darkness', 'year': 2013.0}
{'title': 'Star 

{'title': 'Ot nishto neshto', 'year': 1979.0}
{'title': 'Secret Agent OO Soul', 'year': 1990.0}
{'title': 'OT: Our Town', 'year': 2002.0}
{'title': 'Nyama nishto po-hubavo ot loshoto vreme', 'year': 1971.0}
{'title': 'Oo-lee-byeol il-ho-wa eol-lug-so', 'year': 2014.0}
{'title': 'Rock Oo! Rimba bara kembali', 'year': 2013.0}
{'title': 'Oo na sige na', 'year': 1994.0}
{'title': 'Oo Nina Bobo', 'year': 2014.0}
{'title': 'Ot W Far', 'year': 2015.0}
{'title': 'Oo na... Mahal na kung mahal', 'year': 1999.0}
{'title': 'Viimane öö', 'year': 2003.0}
{'title': "Oo ako'y espiya", 'year': 1946.0}
{'title': 'James Bande OO sexe', 'year': 1982.0}
{'title': 'Öö', 'year': 2007.0}
{'title': 'Los 5 Faust de F.W. Murnau', 'year': 2002.0}
{'title': 'The Other Man: F.W. de Klerk and the End of Apartheid', 'year': 2014.0}
{'title': 'Oy!', 'year': 2009.0}
{'title': 'Oy, vy, gusi...', 'year': 1991.0}
{'title': 'Pif-paf, oy-oy-oy!', 'year': 1980.0}
{'title': 'Oy Vey! My Son Is Gay!!', 'year': 2009.0}
{'title':

{'title': 'KK Family List', 'year': 1997.0}
{'title': 'Km. 0 - Kilometer Zero', 'year': 2000.0}
{'title': '9 KK Road', 'year': 2010.0}
{'title': '7 km da Gerusalemme', 'year': 2007.0}
{'title': 'KK: The Girl with the Dog', 'year': 2013.0}
{'title': 'Who Is KK Downey?', 'year': 2008.0}
{'title': 'KL 10 Patthu', 'year': 2015.0}
{'title': 'Pet filmova o Nives K.K.', 'year': 1980.0}
{'title': 'Nevoljite na pokojniot K.K', 'year': 1963.0}
{'title': 'KK 17', 'year': 1964.0}
{'title': 'Evolusi: KL Drift 2', 'year': 2010.0}
{'title': 'K.K. Steincke', 'year': 1932.0}
{'title': 'KL Gangster 2', 'year': 2013.0}
{'title': 'Kl', 'year': 2011.0}
{'title': 'Iznenadna i prerana smrt pukovnika K.K', 'year': 1987.0}
{'title': 'Evolusi: KL Drift', 'year': 2008.0}
{'title': 'Hund, søndag kl. elleve', 'year': 2004.0}
{'title': 'Väckning kl. 06.00', 'year': 1971.0}
{'title': 'Dir en Grey: [KR] Cube', 'year': 2000.0}
{'title': 'KL Zombi', 'year': 2013.0}
{'title': 'KL Gangster', 'year': 2011.0}
{'title': 'La

In [101]:
#3) Contar las peliculas realizadas entre 1990 y 2000

count=db.movieDetails.count_documents({'$and':[{'year':{'$gte':1990}},{'year':{'$lte':2000}}]})
print(count)

372


In [102]:
#4) ¿Cuáles son los titulos de peliculas que ganaron más de 5 awards?

for i in db.movieDetails.find({'awards.wins':{'$gt':5}},{'title':1,'awards.wins':1,'_id':0}):
    print(i)

{'title': 'West Side Story', 'awards': {'wins': 18.0}}
{'title': 'Wild Wild West', 'awards': {'wins': 10.0}}
{'title': 'How the West Was Won', 'awards': {'wins': 7.0}}
{'title': 'Star Wars: Episode V - The Empire Strikes Back', 'awards': {'wins': 15.0}}
{'title': 'Star Wars: Episode IV - A New Hope', 'awards': {'wins': 38.0}}
{'title': 'Star Wars: Episode I - The Phantom Menace', 'awards': {'wins': 17.0}}
{'title': 'Star Wars: Episode VI - Return of the Jedi', 'awards': {'wins': 15.0}}
{'title': 'Star Wars: Episode III - Revenge of the Sith', 'awards': {'wins': 21.0}}
{'title': 'Star Wars: Episode II - Attack of the Clones', 'awards': {'wins': 13.0}}
{'title': 'Star Trek Into Darkness', 'awards': {'wins': 6.0}}
{'title': 'Star Trek', 'awards': {'wins': 22.0}}
{'title': 'Star Trek: First Contact', 'awards': {'wins': 8.0}}
{'title': 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb', 'awards': {'wins': 13.0}}
{'title': 'Love Actually', 'awards': {'wins': 10.0}}
{'titl

In [107]:
#5) ¿Cuál es el rating promedio en el sitio imdb para todas las peliculas?

for i in db.movieDetails.aggregate([{'$group':{'_id':'null','average':{'$avg':"$imdb.rating"}}}]):
    print(i)

{'_id': 'null', 'average': 6.5031560490940965}


In [108]:
#6) ¿Cuáles son las peliculas en las cuales participó el actor Nicolas Cage?

for i in db.movieDetails.find({'actors':"Nicolas Cage"},{'_id':0,'title':1}):
    print(i)

{'title': 'Red Rock West'}
{'title': 'Wild at Heart'}


In [109]:
#7) ¿Cuál es el número de pelicula por año?

for i in db.movieDetails.aggregate([{'$group':{'_id':"$year",'numeropeliculas':{'$sum':1}}}]):
    print(i)

{'_id': 1927.0, 'numeropeliculas': 1}
{'_id': 1944.0, 'numeropeliculas': 2}
{'_id': 1935.0, 'numeropeliculas': 1}
{'_id': 1903.0, 'numeropeliculas': 2}
{'_id': 2018.0, 'numeropeliculas': 1}
{'_id': 1909.0, 'numeropeliculas': 2}
{'_id': 1932.0, 'numeropeliculas': 3}
{'_id': 1926.0, 'numeropeliculas': 1}
{'_id': 1897.0, 'numeropeliculas': 2}
{'_id': 1917.0, 'numeropeliculas': 1}
{'_id': 1920.0, 'numeropeliculas': 1}
{'_id': 1918.0, 'numeropeliculas': 1}
{'_id': 1937.0, 'numeropeliculas': 3}
{'_id': 1933.0, 'numeropeliculas': 4}
{'_id': 1905.0, 'numeropeliculas': 2}
{'_id': 1936.0, 'numeropeliculas': 2}
{'_id': 1949.0, 'numeropeliculas': 2}
{'_id': 1910.0, 'numeropeliculas': 2}
{'_id': 1939.0, 'numeropeliculas': 3}
{'_id': 1925.0, 'numeropeliculas': 5}
{'_id': 1943.0, 'numeropeliculas': 4}
{'_id': 1945.0, 'numeropeliculas': 2}
{'_id': 1979.0, 'numeropeliculas': 16}
{'_id': 1970.0, 'numeropeliculas': 11}
{'_id': 1978.0, 'numeropeliculas': 8}
{'_id': 1981.0, 'numeropeliculas': 25}
{'_id': 1

In [110]:
#8) ¿Cuál es la pelicula con el mejor rating?

maxTitle=title
maxRating=0

for i in db.movieDetails.find():
    title=i.get('title')
    rating=i.get('imdb')
    rating=rating.get("rating")
    if rating is None:     
        rating=0
    else:
        if(maxRating<rating):
            maxTitle=title
            maxRating=rating
    
print("Titulo de la pelicula: ", maxTitle)
print("Rating de la pelicula: ", maxRating)

Titulo de la pelicula:  DP Fantasies 2
Rating de la pelicula:  9.6


In [111]:
#9) ¿Cuáles son los thrillers con un rating de al meno 6?

for i in db.movieDetails.find({'$and':[{"imdb.rating":{'$gte':6}},{'genres':"Thriller"}]},{'_id':0,'title':1,"imdb.rating":1}):
    print(i)

{'title': 'Red Rock West', 'imdb': {'rating': 7.0}}
{'title': 'Slow West', 'imdb': {'rating': 7.0}}
{'title': 'From Russia with Love', 'imdb': {'rating': 7.5}}
{'title': 'From Paris with Love', 'imdb': {'rating': 6.5}}
{'title': 'Space Cowboys', 'imdb': {'rating': 6.4}}
{'title': "Before the Devil Knows You're Dead", 'imdb': {'rating': 7.3}}
{'title': 'False Trail', 'imdb': {'rating': 6.5}}
{'title': 'Total western', 'imdb': {'rating': 6.4}}
{'title': 'The Man with the Golden Gun', 'imdb': {'rating': 6.8}}
{'title': 'Wild Tales', 'imdb': {'rating': 8.2}}
{'title': 'Law Abiding Citizen', 'imdb': {'rating': 7.4}}
{'title': 'Law of Desire', 'imdb': {'rating': 7.2}}
{'title': 'Beyond the Law', 'imdb': {'rating': 6.5}}
{'title': 'Shutter Island', 'imdb': {'rating': 8.1}}
{'title': 'The Woman in Black', 'imdb': {'rating': 6.4}}
{'title': 'Lost Highway', 'imdb': {'rating': 7.7}}
{'title': 'Angel Heart', 'imdb': {'rating': 7.3}}
{'title': 'Wild at Heart', 'imdb': {'rating': 7.3}}
{'title': 'In

In [112]:
#10) ¿Cuáles son todos los valores de generos posibles?

for i in db.movieDetails.distinct("genres"):
    print(i)

Crime
Drama
Musical
Action
Comedy
Western
Adventure
Animation
Family
Thriller
Fantasy
Documentary
Sci-Fi
Romance
Biography
Mystery
Horror
Music
History
War
Short
Film-Noir
Sport
Talk-Show
News
Reality-TV
Adult
Game-Show


In [113]:
#11) ¿Cuáles son las peliculas que contienen la palabra 'Star'?

for i in db.movieDetails.find({'title': {'$regex':".*star", '$options':"i"}},{'_id':0, 'title':1}):
    print(i)

{'title': 'Star Wars: Episode V - The Empire Strikes Back'}
{'title': 'Star Wars: Episode IV - A New Hope'}
{'title': 'Star Wars: Episode I - The Phantom Menace'}
{'title': 'Star Wars: Episode VI - Return of the Jedi'}
{'title': 'Star Wars: Episode III - Revenge of the Sith'}
{'title': 'Star Wars: Episode II - Attack of the Clones'}
{'title': 'Star Trek Into Darkness'}
{'title': 'Star Trek'}
{'title': 'Star Trek: First Contact'}
{'title': 'Star Trek II: The Wrath of Khan'}
{'title': 'Like Stars on Earth'}
{'title': 'Precure Allstars DX'}
{'title': 'Star Cuts 66: K.C. Valentine'}
{'title': 'JJ Star... How Embarrassing'}
{'title': 'Star Trek VI: The Undiscovered Country'}
{'title': 'RE:AX aka Peace Starts With Me'}
{'title': 'CD Ram: Sex Star Interactive'}
{'title': 'Search for a CW Star'}
{'title': 'Search for a CW Star'}


In [114]:
#12) ¿Cuáles son los títulos que empiezan con la letra 'T'?

for i in db.movieDetails.find({},{'_id':0,'title':1}):
    if(i.get('title')[0]=='T'):
        print(i)

{'title': 'Turks in Space'}
{'title': 'The Adventures of Tintin'}
{'title': 'The Adventures of Priscilla, Queen of the Desert'}
{'title': 'The Adventures of Robin Hood'}
{'title': 'The Many Adventures of Winnie the Pooh'}
{'title': 'The Adventures of Sharkboy and Lavagirl 3-D'}
{'title': 'The Adventures of Buckaroo Banzai Across the 8th Dimension'}
{'title': 'The Extraordinary Adventures of Adèle Blanc-Sec'}
{'title': 'The Adventures of Rocky & Bullwinkle'}
{'title': 'The Truman Show'}
{'title': 'The Rocky Horror Picture Show'}
{'title': 'The Last Picture Show'}
{'title': 'The Greatest Show on Earth'}
{'title': "There's No Business Like Show Business"}
{'title': 'The Evil Dead'}
{'title': 'Treasure Planet'}
{'title': 'The Karate Kid'}
{'title': 'The Karate Kid'}
{'title': 'The Heartbreak Kid'}
{'title': 'The Kid'}
{'title': 'The Kid'}
{'title': 'Trail of the Pink Panther'}
{'title': 'The Kid with a Bike'}
{'title': 'The Karate Kid, Part II'}
{'title': 'The Hallelujah Trail'}
{'title': 

In [115]:
#13) ¿Cuáles son las peliculas que no son de tipo Drama o Thriller?

for i in db.movieDetails.find({'$and':[{'genres':{'$ne':"Thriller"}},{'genres':{'$ne':"Drama"}}]},{'_id':0,'title':1}):
    print(i)

{'title': 'Wild Wild West'}
{'title': 'Once Upon a Time in the West'}
{'title': 'An American Tail: Fievel Goes West'}
{'title': 'How the West Was Won'}
{'title': 'A Million Ways to Die in the West'}
{'title': 'Journey to the West'}
{'title': 'Star Wars: Episode V - The Empire Strikes Back'}
{'title': 'West of Memphis'}
{'title': 'Star Wars: Episode IV - A New Hope'}
{'title': 'Star Wars: Episode I - The Phantom Menace'}
{'title': 'Star Wars: Episode VI - Return of the Jedi'}
{'title': 'Star Wars: Episode III - Revenge of the Sith'}
{'title': 'Star Wars: Episode II - Attack of the Clones'}
{'title': 'Star Trek Into Darkness'}
{'title': 'Star Trek'}
{'title': 'Star Trek: First Contact'}
{'title': 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb'}
{'title': 'I Love You, Man'}
{'title': 'I Love You Phillip Morris'}
{'title': '2001: A Space Odyssey'}
{'title': 'Office Space'}
{'title': 'Zathura: A Space Adventure'}
{'title': 'Plan 9 from Outer Space'}
{'title': 'Muppets

## Ejercicio 4: Conceptos avanzados de Base de Datos: MySQL y Mongo

- <b>Indíce: optimización de consultas</b>

Referencias:
- Indíces en Mongo: https://docs.mongodb.com/manual/indexes/
- Indíces en MySQL: https://dev.mysql.com/doc/refman/8.0/en/optimization-indexes.html
- Introducción a indíces en MySQL: https://www.adictosaltrabajo.com/2015/09/11/introduccion-a-indices-en-mysql/

1) ¿Qué es un Indíce y de qué sirve?

    Es una dirección de memoria que apunta a una tabla o un conjunto de estas

2) ¿Cuándo los SGBDs utilizan los Indíces?

    Los indices se usan cuando se quiere realizar muchas consultas y se quiere optimizar en tiempo

3) ¿Cuáles son los tipos de Indíces en MySQL y Mongo?

    En MySQL existe: PRIMARY KEY | INDEX | UNIQUE | FULL TEXT | SPATIAL
    En Mongo existe: Simple | Compuesto | Unico | Sparse | Hash | Array | Texto | Geoposicion

4) Si los indíces son utiles, ¿por qué no crear indíces por cada columna/atributo? ¿Cuáles son los límimtes?

Porque crear y actualizar un indice es muy costoso en cuanto a tiempo, entonces no siempre es conveniente crear indices

5) Conectarse a la base de datos 'sakila' de MySQL y ejecutar la consulta siguiente para poder conservar información sobre el rendimiento de consultas:

<code>set profiling=1;</code>

Ejecutar 10 veces la consulta siguiente:
<code>select * from rental where return_date>="2005-08-25 17:48:44";</code>

Luego, ejecutar el comando:
<code>show profiles;</code>

¿Cuál es la duración promedia del SGBD para responder a esta consulta?

    Tarda aproximadamente 0.025 segundos
    
6) Crear un index sobre la columna 'return_date', y medir nuevamente 10 veces el rendimiento de la misma consulta. ¿Cuál es el nuevo valor promedio?

    Ahora la duración es de aproximadamente 0.016

7) ¿Cómo crear índices en Mongo?

    db.users.ensureIndex( { "Nombre_atributo" : 1 } )

- <b>Transacciones</b>

Referencias:
- Introducción a Transacciones en MySQL: http://raknarrok.blogspot.com/2011/03/introduccion-transacciones.html
- Transacción en MySQL: https://dev.mysql.com/doc/refman/8.0/en/commit.html
- Transacción en Mongo: https://docs.mongodb.com/manual/core/transactions/

1) ¿Qué es una transacción? 

    Es un conjunto de operaciones elementales que se realiza de forma aislada para no perder consistencia en una base de datos

2) ¿En términos de transacción, cuáles son las características que deberian garantizar un buen SGBD?

    Se debe garantizar que las transacciones sean atomicas (que para finalizar la transaccion, se hayan hecho todas las operaciones, de lo contrario, ninguna operacion se ejecuta) la consistencia de datos al finalizar una transacción, además las inconsistencias en este proceso no se pueden ver en la base de datos mientras ocurre, también se debe garantizar que las transacciones son aisladas, esto quiere decir que si hay operaciones ejecutandose de forma concurrente, no pueden interferirse unas con otras, y por ultimo, la durabilidad de los datos en el sistema, incluso luego de un fallo

3) En MySQL, crear una tabla <b>bank_user(name VARCHAR(10), account INT)</b> que permite conocer el monto disponible en la cuenta bancaria de algunos usuarios.

4) Realizar una operación que permita transferir 100.000 pesos del usuario A al usuario B. ¿Cómo garantizar que la operación no va a generar algún incoherencia en los datos? Escribir el código SQL correspondiente.

    set autocommit=0;
    begin;
    ...
    commit;

5) Supongamos que queremos hacerlo mismo en Mongo. ¿Mongo soporta transacciones ACID? ¿Desde cuándo? ¿Cómo se puede hacer transacciones? ¿Con qué limitaciones?

    La forma de hacer transacciones en mongo es mediante una librería que permita hacerlo, ya que mongo no ha implementado las transacciones dentro de sus facultades

- <b>Sharding y Replicación</b>

1) ¿Cuál es el principio del Sharding?

    Se llama sharding a la practica de particionar una base de datos en diferentes servidores para poder hacer consultas más rápidas

2) ¿Cuál es el principio de la Replicación?

    La replicacion se utiliza para hacer copias de seguridad de una base de datos, de manera que si falla la base de datos principal, puede ser reemplazada por una de estas replicaciones

3) ¿MySQL implementa mecanismos de Sharding/Replicación? y Mongo?

    El mecanismo de replicación si es incorporado por MySql de fabrica, usando copias, donde hay una primaria, y secundarias, tal cual en mongo
    Por otro lado, el sharding no es incorporado por MySql de fabica, pero aún así, si se puede hacer mediante MySql Cluster que puedes particiones automaticamente